# DOS Compliance Testing

The ga4gh-dos-schemas packages includes a number of "compliance tests" - a standardized test harness that can be used to evaluate the compliance of a given service to the Data Object Service schema. The compliance tests can supplement the test suite of a DOS implementation, or can be pointed at a remote DOS endpoint. 

The compliance tests are straightforward to use and can generally be integrated into your project after implementing only one or two methods - one method to make requests to the DOS implementation under test, and another to prepare that implementation for testing.

Let's start with a simple example. If you wanted to test a remote DOS implementation, you could write something like this:

In [1]:
from ga4gh.dos.test.compliance import AbstractComplianceTest
import json
import requests
import unittest

basepath = 'https://dos.commons.ucsc-cgp-dev.org/ga4gh/dos/v1'


class RemoteTest(AbstractComplianceTest):
    @classmethod
    def _make_request(self, meth, path, headers=None, body=None):
        # Where :param:`path` is like `/dataobjects` or `/databundles/{data_bundle_id}`
        # Harcoded access token
        headers = headers or {}
        headers['access_token'] = 'f4ce9d3d23f4ac9dfdc3c825608dc660'
        # Make the request here - :param:`body` is a JSON-formatted string. We
        # convert it here so that Requests will automatically add the right headers.
        r = requests.request(method=meth, url=basepath + path, headers=headers, json=json.loads(body))
        return r.content, r.status_code

To test the implementation at `https://dos.commons.ucsc-cgp-dev.org/`, all we need to do is implement the `_make_request` method that makes an (authenticated) request to the chosen endpoint given a DOS endpoint (such as `/databundles` or `/service-info`), a method, and request content.

Before we run the tests, there's one thing to note - the service that we are testing (dos-azul-lambda dev) does not have data bundle support yet. Luckily, we can pare down what tests we run based on what features the implementation under test supports by setting the `supports` class variable like so:

In [2]:
RemoteTest.supports = ['GetDataObject', 'ListDataObjects', 'UpdateDataObject']

Now, only compliance tests that utilize the `GetDataObject`, `ListDataObjects`, and `UpdateDataObject` DOS operations will run. With that handled, we can run the tests.

`AbstractComplianceTest` subclasses `unittest.TestCase`, so we can run it using a test runner like nose or by using the unittest command hook (e.g. `python -m unittest ...`). For the purposes of running the notebook, we can set up a test suite and run it like so:

In [3]:
import sys
import logging

# First, we need to tweak the normal test logging structure to account
# for the fact we're running in a notebook...
handler = logging.getLogger().handlers[0]
handler.setLevel(logging.WARNING)

# Now, set up the testing harness
suite = unittest.TestLoader().loadTestsFromTestCase(RemoteTest)
# By default, TextTestRunner outputs to sys.stderr.
runner = unittest.TextTestRunner(verbosity=1, stream=sys.stdout)
runner.run(suite)

..s.ss..F.....
FAIL: test_list_data_objects_by_checksum (__main__.RemoteTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/natan/cgl/data-object-service-schemas/python/ga4gh/dos/test/compliance.py", line 55, in wrapper
    return func(self)
  File "/home/natan/cgl/data-object-service-schemas/python/ga4gh/dos/test/compliance.py", line 317, in test_list_data_objects_by_checksum
    self.assertEqual(len(r['data_objects']), 1)
AssertionError: 10 != 1

----------------------------------------------------------------------
Ran 14 tests in 14.933s

FAILED (failures=1, skipped=3)


<unittest.runner.TextTestResult run=14 errors=0 failures=1>

The data in the dev instance can vary, but all the tests that are run should generally pass. Looking at the output, you can see the `s` for tests that were skipped because they weren't specified in the `supports` parameter.

This pattern is easily extended to DOS implementations built on Chalice:

In [4]:
# import chalice
# from my_chalice_app import chalice_app

# We don't have a Chalice app on hand; this code is a non-functional example
# and will fail if run. We set the `chalice` and `my_chalice_app` names so that
# the code runs and doesn't make a fuss...
chalice = {}
my_chalice_app = {}

class ChaliceTest(AbstractComplianceTest):
    @classmethod
    def setUpClass(cls):
        cls.lg = chalice.LocalGateway(chalice_app, chalice.Config())

    @classmethod
    def _make_request(self, meth, path, headers=None, body=None):
        headers = headers or {}
        r = self.lg.handle_request(method=meth, path='/ga4gh/dos/v1' + path,
                                   headers=headers, body=body)
        return r['body'], r['statusCode']

A similar implementation to the above is used in [DataBiosphere/dos-azul-lambda](https://github.com/DataBiosphere/dos-azul-lambda).

This pattern can also be easily extended to test implementations built on Flask. (The reference server included in `ga4gh.dos.server` is built on Connexion, which is itself built on Flask. We test it against the compliance tests in addition to the other integration tests, so it makes a good example. You can check out [Travis](https://travis-ci.org/ga4gh/data-object-service-schemas) to see the latest build results.)

In [5]:
import werkzeug.datastructures

import ga4gh.dos.server
from ga4gh.dos.test.compliance import AbstractComplianceTest

class FlaskTest(AbstractComplianceTest):
    @classmethod
    def setUpClass(cls):
        # :mod:`ga4gh.dos.server` is built on top of :mod:`connexion`,
        # which is built on top of :mod:`flask`, which is built on top
        # of :mod:`werkzeug`, which means we can do some cool nice
        # things with testing.
        app = ga4gh.dos.server.configure_app().app
        cls.client = app.test_client()

        # Populate our new server with some test data objects and bundles
        for data_obj in cls.generate_data_objects(250):
            cls.dos_request('POST', '/dataobjects', body={'data_object': data_obj})
        for data_bdl in cls.generate_data_bundles(250):
            cls.dos_request('POST', '/databundles', body={'data_bundle': data_bdl})

    @classmethod
    def _make_request(cls, meth, path, headers=None, body=None):
        # For documentation on this function call, see
        # :class:`werkzeug.test.EnvironBuilder` and :meth:`werkzeug.test.Client.get`.
        headers = werkzeug.datastructures.Headers(headers)
        r = cls.client.open(method=meth, path='/ga4gh/dos/v1' + path,
                            data=body, headers=headers)
        return r.data, r.status_code

And if we run it, all tests should pass:

In [6]:
import sys
suite = unittest.TestLoader().loadTestsFromTestCase(FlaskTest)
runner.run(suite)

............

ERROR:connexion.decorators.validation:http://localhost/ga4gh/dos/v1/dataobjects/c610f6ec-ce5d-11e8-9acb-484520e6c2c4 validation error: 'data_object' is a required property


..
----------------------------------------------------------------------
Ran 14 tests in 7.360s

OK


<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In conclusion, the compliance tests provide an easy way to supplement existing test suites by testing the compliance of a DOS implementation under test to the Data Object Service schema. This is part of a larger effort to automate the testing and compliance of DOS implementations that is being actively developed that we hope will streamline development and use of DOS.